### Getting differential operators

In [34]:
import torch
import torch.nn as nn
from torch.autograd import grad
import torch.nn.functional as F
import math

torch.set_printoptions(precision=8)

In [35]:
x = torch.ones(2, 2, requires_grad=True)
v = x + 2
y = v ** 2

dy_dx = grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y))
print(f'dy/dx:\n {dy_dx}')

dv_dx = grad(outputs=v, inputs=x, grad_outputs=torch.ones_like(v))
print(f'dv/dx:\n {dv_dx}')

dy/dx:
 (tensor([[6., 6.],
        [6., 6.]]),)
dv/dx:
 (tensor([[1., 1.],
        [1., 1.]]),)


In [36]:
class MyFunction(nn.Module):
    def __init__(self,input_size=2,output_size=1):
        super(MyFunction, self).__init__()
        self.linear1 = nn.Linear(input_size,5)
        self.linear2 = nn.Linear(5,5)
        self.linear3 = nn.Linear(5,output_size)

    def forward(self,x):
        z = torch.sigmoid(self.linear1(x))
        z = torch.sigmoid(self.linear2(z))
        z = torch.sigmoid(self.linear3(z))
        return z

In [39]:
func = MyFunction()

x = torch.rand(8, 2, requires_grad=True)
z = func(x)

dz_dx = grad(outputs=z, inputs=x, grad_outputs=torch.ones_like(z))[0]
print(f'dz/dx:\n {dz_dx}')

dz/dx:
 tensor([[-0.00243671,  0.00142379],
        [-0.00240292,  0.00145341],
        [-0.00237762,  0.00152281],
        [-0.00236594,  0.00129522],
        [-0.00236384,  0.00151623],
        [-0.00237377,  0.00149878],
        [-0.00241358,  0.00146901],
        [-0.00241237,  0.00140220]])


In [38]:
def testm(x):
    y = (x + 1) ** 3
    return y

In [29]:
def gradfn(model, order, xs):
    out = model(xs)
    for i in range(order):
        out = grad(outputs=out, inputs=xs, grad_outputs=torch.ones_like(out), allow_unused=True)[0]
        out.requires_grad = True
    return out

### Using torch.autograd.functional.hessian (for second derivatives)

In [30]:
def quadratic(x):
    y = 3 * x ** 3 + x ** 2 + 4
    return torch.sum(y)

In [31]:
x = torch.tensor([2,5], dtype=torch.float64)

torch.autograd.functional.hessian(quadratic, x, create_graph=True)

tensor([[38.,  0.],
        [ 0., 92.]], grad_fn=<ViewBackward>)

In [54]:
def batch_sec(fn, x, grads):
    out = torch.zeros(x.size()[0], len(grads))

    for i, xi in enumerate(x):
        hess = torch.autograd.functional.hessian(fn, xi, create_graph=True)
        for n, dx in enumerate(grads):
            out[i,n] = hess[dx]

    return out

In [58]:
x = torch.tensor([[1, 2], [3,4], [5, 6], [7, 8]], dtype=torch.float64)

print(batch_sec(quadratic, x, grads=[(0,0), (1,1)]))

tensor([[ 20.,  38.],
        [ 56.,  74.],
        [ 92., 110.],
        [128., 146.]], grad_fn=<CopySlices>)
